In [8]:
# Create the database
# !pip install mysql-connector-python
import mysql.connector
from mysql.connector import Error

try:
    # Establish a connection to the MySQL server (without specifying a database)
    connection = mysql.connector.connect(
        host='seniorscamsafe.cdu48eiyg8cf.ap-southeast-2.rds.amazonaws.com',
        user='admin',
        password='MonashTA0620241!'
    )
    
    if connection.is_connected():
        cursor = connection.cursor()
        cursor.execute("CREATE DATABASE SeniorSafeScam")
        print("Database 'SeniorSafeScam' created successfully")

except Error as e:
    print(f"Error: {e}")

finally:
    if connection.is_connected():
        cursor.close()
        connection.close()
        print("MySQL connection is closed")



Database 'SeniorSafeScam' created successfully
MySQL connection is closed


In [1]:
# Test connection to database
import mysql.connector
from mysql.connector import Error

# Establish the connection
try:
    connection = mysql.connector.connect(
        host='seniorscamsafe.cdu48eiyg8cf.ap-southeast-2.rds.amazonaws.com',
        database='SeniorSafeScam',
        user='admin',
        password='MonashTA0620241!'
    )
    
    if connection.is_connected():
        print("Successfully connected to the database")
except Error as e:
    print(f"Error: {e}")


ModuleNotFoundError: No module named 'mysql'

# Create Table for Scam Watch Data
USE SeniorSafeScam;

CREATE TABLE ScamWatch (
    
    StartOfMonth DATE,
    Address_State VARCHAR(100),
    Scam_Contact_Mode VARCHAR(100),
    Complainant_Age VARCHAR(50),
    Complainant_Gender VARCHAR(10),
    Category_Level_2 VARCHAR(255),
    Category_Level_3 VARCHAR(255),
    Amount_lost DECIMAL(18, 2),
    Number_of_reports INT
);

In [18]:
import pymysql
import pandas as pd

# Database connection
connection = pymysql.connect(
    host='seniorscamsafe.cdu48eiyg8cf.ap-southeast-2.rds.amazonaws.com',
    user='admin',  
    password='MonashTA0620241!',  
    database='SeniorSafeScam',
    port=3306
)

# Path to your CSV file
csv_file_path = '/Users/lukassteel/Desktop/SeniorScamSafe/RawData/ScamWatch/Scamwatch901 Public Scams Dashboard - Export.csv'

# Load CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Print the columns to ensure they match your table
print(df.columns)

# Replace NaNs with None
df = df.where(pd.notnull(df), None)

# Function to clean the 'Amount_lost' column
def clean_amount_lost(value):
    try:
        # Remove $ and commas, and convert to float
        return float(value.replace('$', '').replace(',', ''))
    except (ValueError, AttributeError):
        # If conversion fails, return None
        return None

# Apply the cleaning function to the 'Amount_lost' column
df['Amount_lost'] = df['Amount_lost'].apply(clean_amount_lost)

# Ensure column names match those in your MySQL table
df.columns = [
    'StartOfMonth', 'Address_State', 'Scam_Contact_Mode', 'Complainant_Age',
    'Complainant_Gender', 'Category_Level_2', 'Category_Level_3',
    'Amount_lost', 'Number_of_reports'
]

# Convert DataFrame to list of tuples, ensuring None values replace NaNs
data = [tuple(None if pd.isna(item) else item for item in row) for row in df.values]

# Quote column names using backticks
columns = ', '.join([f'`{col}`' for col in df.columns])
placeholders = ', '.join(['%s'] * len(df.columns))

# Generate the SQL query for inserting data
sql = f"INSERT INTO `ScamWatch` ({columns}) VALUES ({placeholders})"

try:
    with connection.cursor() as cursor:
        # Execute the SQL command to insert data
        cursor.executemany(sql, data)
        # Commit changes
        connection.commit()
        print("Data uploaded successfully.")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    connection.close()


Index(['StartOfMonth', 'Address_State', 'Scam___Contact_Mode',
       'Complainant_Age', 'Complainant_Gender', 'Category_Level_2',
       'Category_Level_3', 'Amount_lost', 'Number_of_reports'],
      dtype='object')
Data uploaded successfully.


In [21]:
# Test to see data was put into table
import pymysql
import pandas as pd

# Database connection
connection = pymysql.connect(
    host='seniorscamsafe.cdu48eiyg8cf.ap-southeast-2.rds.amazonaws.com',
    user='admin',  
    password='MonashTA0620241!',  
    database='SeniorSafeScam',
    port=3306
)

try:
    with connection.cursor() as cursor:
        # SQL query to select the top 10 records based on Amount_lost
        sql = """
        SELECT * FROM ScamWatch
        ORDER BY Amount_lost DESC
        LIMIT 10;
        """
        
        # Execute the query
        cursor.execute(sql)
        
        # Fetch the results
        results = cursor.fetchall()
        
        # Get column names from cursor
        column_names = [i[0] for i in cursor.description]
        
        # Convert the results to a pandas DataFrame
        df = pd.DataFrame(results, columns=column_names)
        
        # Display the DataFrame
        print(df)

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    connection.close()


  StartOfMonth      Address_State           Scam_Contact_Mode Complainant_Age  \
0   2024-03-01    New South Wales                  Phone call     65 and over   
1   2023-02-01  Western Australia                       Email     65 and over   
2   2023-04-01         Queensland                    Internet     65 and over   
3   2020-03-01           Victoria                  Phone call     65 and over   
4   2021-12-01         Queensland                  Phone call     65 and over   
5   2023-05-01         Queensland                  Phone call     65 and over   
6   2023-12-01           Victoria  Social media/Online forums     65 and over   
7   2024-06-01    New South Wales                  Phone call     65 and over   
8   2022-07-01    South Australia                       Email     65 and over   
9   2022-12-01         Queensland                   In person     65 and over   

  Complainant_Gender                            Category_Level_2  \
0             Female                    

In [25]:
import pymysql
import pandas as pd

# Database connection
connection = pymysql.connect(
    host='seniorscamsafe.cdu48eiyg8cf.ap-southeast-2.rds.amazonaws.com',
    user='admin',  
    password='MonashTA0620241!',  
    database='SeniorSafeScam',
    port=3306
)

# Path to your CSV file
csv_file_path = '/Users/lukassteel/Desktop/SeniorScamSafe/RawData/PhishingEmailDetection/Phishing_Email.csv'

# Load CSV file into a pandas DataFrame
df = pd.read_csv(csv_file_path)

# Rename columns to match the table schema, including the ID column
df.columns = ['ID', 'EmailText', 'EmailType']

# Replace NaNs with None
df = df.where(pd.notnull(df), None)

# Remove the ID column if letting MySQL auto-increment it
df = df.drop(columns=['ID'])

# Convert DataFrame to list of tuples, ensuring None values replace NaNs
data = [tuple(None if pd.isna(item) else item for item in row) for row in df.values]

# Quote column names using backticks, excluding the ID column
columns = ', '.join([f'`{col}`' for col in df.columns])
placeholders = ', '.join(['%s'] * len(df.columns))

# Generate the SQL query for inserting data, excluding the ID column
sql = f"INSERT INTO `PhishingScams` ({columns}) VALUES ({placeholders})"

try:
    with connection.cursor() as cursor:
        # Create the PhishingScams table if it doesn't exist
        create_table_query = """
        CREATE TABLE IF NOT EXISTS `PhishingScams` (
            `ID` INT AUTO_INCREMENT PRIMARY KEY,
            `EmailText` TEXT,
            `EmailType` VARCHAR(50)
        );
        """
        cursor.execute(create_table_query)
        connection.commit()

        # Execute the SQL command to insert data
        cursor.executemany(sql, data)
        # Commit changes
        connection.commit()
        print("Data uploaded successfully.")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    connection.close()


Data uploaded successfully.


In [31]:
# Test to see data was put into table
import pymysql
import pandas as pd

# Database connection
connection = pymysql.connect(
    host='seniorscamsafe.cdu48eiyg8cf.ap-southeast-2.rds.amazonaws.com',
    user='admin',  
    password='MonashTA0620241!',  
    database='SeniorSafeScam',
    port=3306
)

try:
    with connection.cursor() as cursor:
        # SQL query to select the top 10 records
        sql = """
        SELECT * FROM PhishingScams
        ORDER BY ID
        ASC
        LIMIT 10;
        """
        
        # Execute the query
        cursor.execute(sql)
        
        # Fetch the results
        results = cursor.fetchall()
        
        # Get column names from cursor
        column_names = [i[0] for i in cursor.description]
        
        # Convert the results to a pandas DataFrame
        df = pd.DataFrame(results, columns=column_names)
        
        # Display the DataFrame
        print(df)

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    connection.close()


   ID                                          EmailText       EmailType
0   1  re : 6 . 1100 , disc : uniformitarianism , re ...      Safe Email
1   2  the other side of * galicismos * * galicismo *...      Safe Email
2   3  re : equistar deal tickets are you still avail...      Safe Email
3   4  \nHello I am your hot lil horny toy.\n    I am...  Phishing Email
4   5  software at incredibly low prices ( 86 % lower...  Phishing Email
5   6  global risk management operations sally congra...      Safe Email
6   7  On Sun, Aug 11, 2002 at 11:17:47AM +0100, wint...      Safe Email
7   8  entourage , stockmogul newsletter ralph velez ...  Phishing Email
8   9  we owe you lots of money dear applicant , afte...  Phishing Email
9  10  re : coastal deal - with exxon participation u...      Safe Email


In [33]:
# Create Map Table
import pymysql
import json

# Database connection
connection = pymysql.connect(
    host='seniorscamsafe.cdu48eiyg8cf.ap-southeast-2.rds.amazonaws.com',
    user='admin',
    password='MonashTA0620241!',
    database='SeniorSafeScam',
    port=3306
)

# Path to your JSON file
json_file_path = '/Users/lukassteel/Desktop/SeniorScamSafe/RawData/Map/australian-states.json'

# Load JSON data
with open(json_file_path, 'r') as file:
    data = json.load(file)

try:
    with connection.cursor() as cursor:
        # Step 1: Create the map table
        create_table_query = """
        CREATE TABLE IF NOT EXISTS map (
            id INT PRIMARY KEY,
            state_code VARCHAR(10),
            state_name VARCHAR(100),
            geometry_type VARCHAR(50),
            coordinates JSON
        );
        """
        cursor.execute(create_table_query)
        connection.commit()

        # Step 2: Insert data into the map table
        insert_query = """
        INSERT INTO map (id, state_code, state_name, geometry_type, coordinates)
        VALUES (%s, %s, %s, %s, %s)
        """

        for feature in data['features']:
            feature_id = feature['id']
            state_code = feature['properties']['STATE_CODE']
            state_name = feature['properties']['STATE_NAME']
            geometry_type = feature['geometry']['type']
            coordinates = json.dumps(feature['geometry']['coordinates'])

            cursor.execute(insert_query, (feature_id, state_code, state_name, geometry_type, coordinates))

        # Commit changes
        connection.commit()
        print("Data uploaded successfully to the map table.")

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    connection.close()


Data uploaded successfully to the map table.


In [34]:
# Test Map Table

import pymysql
import pandas as pd

# Database connection
connection = pymysql.connect(
    host='seniorscamsafe.cdu48eiyg8cf.ap-southeast-2.rds.amazonaws.com',
    user='admin',
    password='MonashTA0620241!',
    database='SeniorSafeScam',
    port=3306
)

try:
    with connection.cursor() as cursor:
        # SQL query to select the top 10 records
        sql = "SELECT * FROM map LIMIT 10;"
        
        # Execute the query
        cursor.execute(sql)
        
        # Fetch the results
        results = cursor.fetchall()
        
        # Get column names from cursor
        column_names = [i[0] for i in cursor.description]
        
        # Convert the results to a pandas DataFrame
        df = pd.DataFrame(results, columns=column_names)
        
        # Display the DataFrame
        print(df)

except Exception as e:
    print(f"An error occurred: {e}")
finally:
    connection.close()


   id state_code                    state_name geometry_type  \
0   0          1               New South Wales       Polygon   
1   1          2                      Victoria       Polygon   
2   2          3                    Queensland  MultiPolygon   
3   3          4               South Australia  MultiPolygon   
4   4          5             Western Australia       Polygon   
5   5          6                      Tasmania  MultiPolygon   
6   6          7            Northern Territory  MultiPolygon   
7   7          8  Australian Capital Territory       Polygon   

                                         coordinates  
0  [[[149.976679, -37.50506], [148.194946, -36.79...  
1  [[[140.965735, -38.055989], [140.963616, -33.9...  
2  [[[[153.476931, -28.15702], [153.108829, -28.3...  
3  [[[[129.00134, -31.687068], [129.001854, -25.9...  
4  [[[129.001242, -14.879473], [129.00134, -31.68...  
5  [[[[144.754462, -41.061823], [144.686043, -40....  
6  [[[[138.00118, -16.544509], [137.99

In [2]:
import pymysql

# Database connection
connection = pymysql.connect(
    host='seniorscamsafe.cdu48eiyg8cf.ap-southeast-2.rds.amazonaws.com',
    user='admin',  
    password='MonashTA0620241!',  
    database='SeniorSafeScam',
    port=3306
)

try:
    with connection.cursor() as cursor:
        # SQL query to delete records with dates after 2024
        delete_query = """
        DELETE FROM `ScamWatch` 
        WHERE `StartOfMonth` > '2024-12-31';
        """
        # Execute the delete query
        cursor.execute(delete_query)
        # Commit the changes
        connection.commit()
        print("Records deleted successfully.")
except Exception as e:
    print(f"An error occurred: {e}")
finally:
    connection.close()


Records deleted successfully.
